***IMPORTS Y ACCESO A DATOS DE CSV BICIS***

In [1]:
#Importación de paquetes y lectura de csv:
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import random
from sklearn import model_selection
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr
import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


dataBicis = pd.read_csv('hour.csv')
dataBicis = dataBicis.drop("dteday", axis=1)
dataBicis = dataBicis.drop("instant", axis=1)
atributosBicis = dataBicis.loc[:, 'season':'registered']
objetivoBicis = dataBicis['cnt']

(atributos_entrenamientoBicis, atributos_pruebaBicis,
 objetivo_entrenamientoBicis, objetivo_pruebaBicis) = model_selection.train_test_split(
    atributosBicis, objetivoBicis, test_size=.33)

atributos_entrenamientoBicis = atributos_entrenamientoBicis.values
objetivo_entrenamientoBicis = objetivo_entrenamientoBicis.values
atributos_pruebaBicis = atributos_pruebaBicis.values
objetivo_pruebaBicis= objetivo_pruebaBicis.values

rangosBicis = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in dataBicis.columns:
    valor_min = dataBicis[columna].min()
    valor_max = dataBicis[columna].max()
    rangosBicis[columna] = {'min': valor_min, 'max': valor_max}

***MÉTODO LIME***

In [2]:
def metodo_lime(N,x,k,model,rangos):
    random.seed(1)
    Xprima=[]
    R=[]
    W=[]
    Yprima= []
    for i in range (0,N):
        atributosAPerturbar=seleccionar_atributos_aleatorios(x,k)
        xprima = perturbar_muestra(x,atributosAPerturbar,rangos)
        w = distancia_entre_muestras(x,xprima)
        r=atributosAPerturbar
        Xprima.append(xprima)
        R.append(r)
        W.append(w)
        yprima = prediccion_de_modelo(xprima,model)
        Yprima.append(yprima)
    W=np.array(W)
    W = W.reshape(W.shape[0])
    G = entrenar_modelo_ridge(R,Yprima,W)
    return G.coef_

#Se seleccional los k atributos aleatorios a modificar

def seleccionar_atributos_aleatorios(dataframe, k):
    columnas = dataframe.columns.tolist()
    seleccionados = random.sample(range(len(columnas)), k)
    lista = [0] * len(columnas)
    for i in seleccionados:
        lista[i] = 1
    #print(lista)
    return lista

#Se perturba la muestra original de forma que los atributos seleccionados pueden variar en un rango en función del valor máximo y mínimo de dicho atributo
def perturbar_muestra(x, atributosAPerturbar,rangos):
    xprima = x.copy()
    for i in range(0,len(atributosAPerturbar)):
        if atributosAPerturbar[i] == 1:
            rango_min = xprima[i] - rangos[list(rangos.keys())[i]]['min']
            rango_max = rangos[list(rangos.keys())[i]]['max'] - xprima[i]
            xprima[i] = random.uniform(rango_min, rango_max)
    return xprima

# Se calcula la distancia coseno entre la muestra original y la perturbada
def distancia_entre_muestras(x,xprima):
    return cosine_distances(x.values, xprima.values)

#Se predice el resultado de la muestra con el modelo elegido
def prediccion_de_modelo(xprima,model):
    if not isinstance(model, (RandomForestRegressor, RandomForestClassifier)):
        xprima = DMatrix(xprima)
    y_pred = model.predict(xprima)
    return y_pred


#Se entrena el modelo de regresión Ridge con las muestras perturbadas y los pesos calculados
def entrenar_modelo_ridge(R, Yprima, W):
    modelo_entrenado = Ridge(alpha=1.0) 
    modelo_entrenado.fit(R, Yprima, sample_weight=W)
    return modelo_entrenado

***MODELOS BICIS***

In [3]:
#RandomForest
#Creamos el modelo RandomForest (En este caso es Regressor ya que se trata de una tarea de regresión)
modelRandomForestBicis = RandomForestRegressor(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestBicis.fit(atributos_entrenamientoBicis, objetivo_entrenamientoBicis)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainBicis = xgb.DMatrix(atributos_entrenamientoBicis, label=objetivo_entrenamientoBicis)
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
# Definimos los parámetros del modelo XGBoost
paramsBicis = {
    'objective': 'reg:squarederror',  # Objetivo para regresión
    'eval_metric': 'rmse'  # Métrica de evaluación, error cuadrático medio (RMSE)
}

# Entrenamos el modelo XGBoost
modelXGBoostBicis = xgb.train(paramsBicis, dtrainBicis)

In [4]:
prediccionesBicisXG = modelXGBoostBicis.predict(dtestBicis)
explicacionesBicisXG = []
atributosAcambiar = 5
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelXGBoostBicis,rangosBicis)
    explicacionesBicisXG.append(explicacion)
print(explicacionesBicisXG)

[array([[ -52.64336398,  -44.75101664,  -18.27117717,  -55.79439791,
          -2.94358454, -123.80492976,   80.63634961,   61.43975733,
           8.25928909,  -58.27054809,  -41.8074321 ,   56.27589992,
          48.65011095,  143.02504329]]), array([[  1.84196417,  -7.86163154,  -0.10048797,  -0.32119602,
         -0.01251757, -11.11151026,   4.23670387,   6.97529436,
          2.49629259,  -2.01961904,  -7.84911397,   2.0116453 ,
          0.49763403,  11.21654206]]), array([[ 5.52101001, -6.21172287, -0.07224177, -0.43007046, -0.01974083,
        -5.4920463 ,  4.46729371,  1.09699435,  1.59583751,  3.39987118,
        -6.19198204, -3.40931167,  0.17152077,  5.57458841]]), array([[-24.19701245, -31.47447086,  -4.39018468,  -9.6190745 ,
         -0.55121461, -89.9874186 ,  51.44852939,  42.92907389,
         -6.89935416,  -9.3121779 , -30.92325626,   9.38201082,
          8.59589911,  94.99865081]]), array([[ 1.53487659, -6.29098776, -0.04533446, -0.03986891, -0.01038585,
        -5

In [5]:
prediccionesBicisRF = modelRandomForestBicis.predict(atributos_pruebaBicis)
explicacionesBicisRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestBicis,rangosBicis)
    explicacionesBicisRF.append(explicacion)
print(explicacionesBicisRF)

[array([[ -47.81007793,  -47.94036299,  -18.90372771,  -56.36367873,
          -3.37159413, -128.21533024,   88.50908526,   58.6099727 ,
          -3.63455332,  -49.32186638,  -44.56876886,   47.01158653,
          57.81894357,  148.18037223]]), array([[  3.53296177, -10.06791116,  -0.10956303,  -0.34709801,
         -0.01330703, -10.32596814,   3.62046918,   6.815062  ,
          5.97869405,  -3.16885274, -10.05460414,   3.16043397,
          0.53926387,  10.44041943]]), array([[ 6.03272308, -6.81596564, -0.07716369, -0.40736976, -0.02078682,
        -4.26544135,  3.15577399,  1.18683105,  3.23858696,  2.02243995,
        -6.79517882, -2.03238266,  0.42448455,  4.35344916]]), array([[-23.11012598, -32.17583965,  -4.48869434, -10.32109811,
         -0.5698408 , -91.63965948,  53.06704462,  43.06130921,
         -8.53649915,  -7.90102915, -31.60599885,   7.97617696,
          9.47091229,  96.77334243]]), array([[ 1.62687073, -6.22158042, -0.04706782, -0.04166019, -0.01079304,
        -5

***VALIDACIÓN DE MODELO***

In [6]:
def evaluate_model(model, Xentrenamiento, yentrenamiento, Xtest, ytest):
    # Validación cruzada
    cv_scores = cross_val_score(model, Xentrenamiento, yentrenamiento, cv=2)
    mean_cv_score = np.mean(cv_scores)
    
    print("Puntuación media de validación cruzada:", mean_cv_score)
    
    #.score
    score = model.score(Xtest, ytest)
    print("Puntuación en el conjunto de prueba:", score)

In [7]:
evaluate_model(modelRandomForestBicis, atributos_entrenamientoBicis, objetivo_entrenamientoBicis, atributos_pruebaBicis, objetivo_pruebaBicis)

Puntuación media de validación cruzada: 0.9994244931531868
Puntuación en el conjunto de prueba: 0.9997667610392554


******

In [8]:


nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoBicis).iloc[3]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)


Iterac=10
atributosAcambiar=5

nuevos_ejemplos = pd.DataFrame([[1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7]])
print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestBicis))
print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostBicis))
print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestBicis,rangosBicis))

print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostBicis,rangosBicis))

    0    1    2    3    4    5    6    7    8       9     10      11   12  \
3  1.0  1.0  1.0  8.0  0.0  1.0  1.0  3.0  0.2  0.2121  0.86  0.1343  5.0   

      13  
3  267.0  
Prediccion RF [272.84]
Prediccion XG [270.35678]
Método LIME con Random Forest:  [[  2.87128795 -11.48198534  -0.17663304  -0.15601571  -0.02922231
  -12.09883548   3.5249329    8.75053562   7.79420349  -5.11646613
  -11.45276304   5.09080665   0.2011231   12.27903134]]
Método LIME con XGBoost:  [[  1.60094731 -10.23982509  -0.15949971  -0.14036036  -0.02660389
  -12.27051301   4.75066046   7.67935226   6.4276553   -3.91579558
  -10.21322119   3.89243678   0.18150892  12.43325781]]


******

***

***ACCESO A DATOS DE CSV PEN***

In [9]:

# Cargar el archivo CSV
pen = pd.read_csv('pendigits.csv', header=0, names=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14','a15','a16',"class-objective"])

atributosPen = pen.loc[:, 'a1':'a16']
objetivoPen = pen['class-objective']

rangosPen = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in pen.columns:
    valor_min = pen[columna].min()
    valor_max = pen[columna].max()
    rangosPen[columna] = {'min': valor_min, 'max': valor_max}
    


In [10]:
(atributos_entrenamientoPen, atributos_pruebaPen,
 objetivo_entrenamientoPen, objetivo_pruebaPen) = model_selection.train_test_split(
    atributosPen, objetivoPen, test_size=.33,stratify=objetivoPen)

atributos_entrenamientoPen = atributos_entrenamientoPen.values
objetivo_entrenamientoPen = objetivo_entrenamientoPen.values
atributos_pruebaPen = atributos_pruebaPen.values
objetivo_pruebaPen= objetivo_pruebaPen.values

print(atributos_entrenamientoPen)

[[ 50  81  74 ...  23  84   0]
 [ 22  61  54 ...   1 100   0]
 [ 19  86  63 ...   0   0   2]
 ...
 [ 20  84  60 ...   5   0   0]
 [  0  67  38 ...  90  58 100]
 [100 100  64 ...  21  44   0]]


***MODELOS PEN***

In [11]:

modelRandomForestPen = RandomForestClassifier(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestPen.fit(atributos_entrenamientoPen, objetivo_entrenamientoPen)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainPen = xgb.DMatrix(atributos_entrenamientoPen, label=objetivo_entrenamientoPen)
dtestPen = xgb.DMatrix(atributos_pruebaPen)

# Definimos los parámetros del modelo XGBoost
paramsPen = {
    'objective': 'multi:softmax',  # Función objetivo para clasificación multiclase
    'num_class': 10,       # Número de clases en tu problema
    'eval_metric': 'merror' 
}

# Entrenamos el modelo XGBoost
modelXGBoostPen = xgb.train(paramsPen, dtrainPen)

In [12]:
prediccionesPenXG = modelXGBoostPen.predict(dtestPen)
explicacionesPenXG = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelXGBoostPen, rangosPen)
    explicacionesPenXG.append(explicacion)
print(explicacionesPenXG)

[array([[-0.03967286, -0.18986797, -0.01460489, -0.0301292 , -0.20159361,
        -0.01016604,  0.16987814, -0.00026996,  0.18788106, -0.00581225,
         0.04548511,  0.03233786, -0.125842  , -0.03769544, -0.02602272,
         0.24609478]]), array([[-0.08810871, -0.19067656, -0.01971515, -0.05917688, -0.21638866,
        -0.02334347,  0.23729934, -0.01791093,  0.20899608,  0.08702607,
         0.00108264, -0.02649903, -0.15708704,  0.01183772, -0.0769471 ,
         0.32961168]]), array([[-0.25575719, -0.04789702, -0.08172292, -0.20114123, -0.12093562,
        -0.04628674,  0.3856174 ,  0.00851736,  0.02236611,  0.42066426,
        -0.16490707, -0.273011  , -0.04338722,  0.15025172, -0.19991102,
         0.44754018]]), array([[-0.12955951,  0.04087596,  0.05862097, -0.06186966,  0.10912413,
        -0.06330243, -0.06942857, -0.32873799,  0.27384741,  0.09599918,
         0.03356033, -0.04408298, -0.25902379,  0.1730306 ,  0.10099951,
         0.06994685]]), array([[-0.22741208, -0.045

In [13]:
prediccionesPenRF = modelRandomForestPen.predict(atributos_pruebaPen)
explicacionesPenRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestPen, rangosPen)
    explicacionesPenRF.append(explicacion)
print(explicacionesPenRF)

[array([[ 0.09941117, -0.15621066,  0.01633668,  0.06604476, -0.13322399,
         0.03161184,  0.07311131,  0.03356587,  0.11774937, -0.12927216,
         0.02986099,  0.06186724, -0.10879096, -0.08674306,  0.04878467,
         0.03589691]]), array([[-0.03234024, -0.17541444, -0.00935472, -0.0228669 , -0.18845669,
        -0.00415294,  0.22194804, -0.00033248,  0.17411406,  0.06347725,
        -0.03113701, -0.03881174, -0.18541375,  0.03239629, -0.03124671,
         0.227592  ]]), array([[-0.38355799, -0.17039518, -0.29484717, -0.11033912, -0.36274469,
        -0.41224192, -0.0334608 , -0.17445498,  0.38137554,  0.08207914,
         0.30147885,  0.53522853, -0.25835926,  0.57516076, -0.05041232,
         0.37549059]]), array([[-0.01428637,  0.027504  ,  0.03273751,  0.12152988,  0.06342181,
        -0.13237363,  0.04018826, -0.12491338,  0.09078647, -0.01504622,
         0.02933259, -0.10705269, -0.07974411,  0.0385928 ,  0.06083096,
        -0.03150787]]), array([[-0.53257088, -0.083

In [14]:
nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoPen).iloc[2]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)
Iterac=10
atributosAcambiar=5


print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestPen))
print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostPen))
print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestPen,rangosPen))
print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostPen,rangosPen))

   0   1   2    3   4   5   6   7   8   9    10  11  12  13  14  15
2  19  86  63  100  89  80  51  57  72  37  100  14  49   0   0   2
Prediccion RF [3]
Prediccion XG [3.]
Método LIME con Random Forest:  [[-0.25731004  0.11545116 -0.16903196 -0.10925748  0.01799578 -0.30772345
  -0.12151566 -0.15687698  0.12952013  0.19767069  0.05963935  0.26319077
  -0.21458885  0.59652735 -0.04084162 -0.00284919]]
Método LIME con XGBoost:  [[-0.0492579  -0.07984904  0.06077093 -0.09732307 -0.05485016  0.12509376
   0.31243551  0.05049567 -0.01190317  0.18015822 -0.13090032 -0.33461395
   0.03765609 -0.20327493 -0.09039781  0.28576017]]


***VALIDACIÓN***

In [15]:
evaluate_model(modelRandomForestPen, atributos_entrenamientoPen, objetivo_entrenamientoPen, atributos_pruebaPen, objetivo_pruebaPen)

Puntuación media de validación cruzada: 0.9832669322709164
Puntuación en el conjunto de prueba: 0.9915082895268904


***MÉTRICAS***

In [64]:
predicciones = prediccionesBicisRF[:256]
objetivos = objetivo_pruebaBicis[:256]
explicaciones = explicacionesBicisRF

# Identidad

def identidad(i):
    p = predicciones[i]
    e = explicaciones[i]
    return np.linalg.norm(p - p) == 0 and np.linalg.norm(e - e) == 0

# Separabilidad

def separabilidad(i,j):
    p1 = predicciones[i]
    e1 = explicaciones[i]
    p2 = predicciones[j]
    e2 = explicaciones[j]
    return np.linalg.norm(p1 - p2) != 0 and np.linalg.norm(e1 - e2) > 0 

# Estabilidad

def estabilidad(i):
    p = predicciones[i]
    e = explicaciones[i]
    d_muestras = np.array([np.linalg.norm(p - x) for x in predicciones])
    d_explicaciones = np.array([np.linalg.norm(e - x) for x in explicaciones])
    return spearmanr(d_muestras, d_explicaciones)[0] > 0

# Selectividad

def selectividad(i):
    return null

# Coherencia
def coherencia(i):
    error_prediccion_original = predicciones[i] - objetivos[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - objetivos[i]
    return np.abs(error_prediccion_original - error_prediccion_nuevo)

# Completitud

def completitud(i):
    error_prediccion_original = predicciones[i] - objetivos[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - objetivos[i]
    return error_prediccion_nuevo / error_prediccion_original

# Congruencia
def congruencia():
    alpha = np.sum(coherencia(i) for i in range(len(predicciones))) / len(predicciones)
    return np.sqrt(np.sum(np.power(coherencia(i) - alpha),2) for i in range(len(predicciones))) / len(predicciones)

In [74]:
# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i)
print("Identidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1)
print("Separabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i)
print("Estabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

for i in range(0, 256):
    print (i, coherencia(i))
    
# Completitud

for i in range(0, 256):
    print (i, completitud(i))

# Congruencia

print (congruencia())

Identidad: True
Separabilidad: True
Estabilidad: False


NameError: name 'predicciones_actualizadas' is not defined